## FASE 1

In [2]:
# importo librerias

import requests
import pandas as pd
import json
import mysql.connector
import numpy as np

In [3]:
# me descargo la info desde la url que nos han dado

url_peliculas="https://beta.adalab.es/resources/apis/pelis/pelis.json"
respuesta=requests.get(url_peliculas)
datos_peliculas=respuesta.json()
df_peliculas=pd.json_normalize(datos_peliculas)
print("Datos descargadosy normalizados en un Data Frame")
print (df_peliculas.head())


Datos descargadosy normalizados en un Data Frame
   id                 titulo   año  duracion  genero  adultos    subtitulos
0   1          The Godfather  1972       175  Crimen    False      [es, en]
1   2  The Godfather Part II  1974       202  Crimen    False      [es, en]
2   3           Pulp Fiction  1994       154  Crimen     True      [es, en]
3   4           Forrest Gump  1994       142   Drama    False  [es, en, fr]
4   5        The Dark Knight  2008       152  Acción    False      [es, en]


In [4]:
df_peliculas.info

<bound method DataFrame.info of      id                                 titulo   año  duracion    genero  \
0     1                          The Godfather  1972       175    Crimen   
1     2                  The Godfather Part II  1974       202    Crimen   
2     3                           Pulp Fiction  1994       154    Crimen   
3     4                           Forrest Gump  1994       142     Drama   
4     5                        The Dark Knight  2008       152    Acción   
..  ...                                    ...   ...       ...       ...   
95   96                        La vita è bella  1997       116     Drama   
96   97                    Requiem for a Dream  2000       102     Drama   
97   98                                Memento  2000       113  Thriller   
98   99  Eternal Sunshine of the Spotless Mind  2004       108     Drama   
99  100                           Donnie Darko  2001       113     Drama   

    adultos    subtitulos  
0     False      [es, en]  

## FASE 2

Hasta aqui me he descargado el .json y lo tengo en un data frame ahora entramos en la parte de mySQL para crear la BD

In [5]:
import mysql.connector
try:
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='?10101948SanMartin',
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)


Conexión exitosa


In [6]:
try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS Adalab_Peliculas"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

Query exitosa


## FASE 3

# crear tabla en my SQL Workbench

In [7]:
mycursor.execute("USE Adalab_Peliculas")

query = '''CREATE TABLE IF NOT EXISTS peliculas (
    id INT PRIMARY KEY AUTO_INCREMENT,
    Titulo VARCHAR(255) NOT NULL,
    Año_Lanzamiento INT,
    Duración INT,
    Genero VARCHAR (255),
    Contenido_adultos BOOLEAN
);'''

mycursor.execute(query)

# introducir datos en tabla creada peliulas

In [ ]:
# Query de inserción
query_insert = """
INSERT INTO peliculas (Titulo, Año_Lanzamiento, Duración, Genero, Contenido_adultos) 
                              
VALUES (%s, %s, %s, %s, %s)
"""


# Convertir DataFrame a lista de tuplas
#pd.notna(): Verifica si el valor no es NaN/null antes de insertarlo
#iterrows(): Itera sobre las filas del DataFrame

# Reemplazar NaN con None para que se inserten como NULL en la BD
# Reemplazar NaN con None de forma más robusta
df_clean = df_peliculas.replace({np.nan: None, 'nan': None, 'NaN': None})
# Convertir DataFrame directamente a lista de tuplas
datos_a_insertar= []
for index,row in df_peliculas.iterrows():
    
    tupla_datos=(
        row ['titulo'], 
        row ['año'],
        row ['duracion'],
        row ['genero'],
        bool(row ['adultos'])
    )
    datos_a_insertar.append(tupla_datos)
# Ejecutar inserción
try:
    mycursor.executemany(query_insert, datos_a_insertar)
    cnx.commit()
    print(f"{mycursor.rowcount} registros insertados")
except Error as e:
    print(f"Error al ejecutar la inserción:{e}")


100 registros insertados


## FASE 4

# He creado las consultas en SQL y las traigo aqui de manera automática 

In [ ]:
import pandas as pd
import mysql.connector
import re 
from mysql.connector import Error

# --- 1. CONFIGURACIÓN ---
DB_CONFIG = {
    'host': '127.0.0.1',
    'database': 'Adalab_Peliculas', 
    'user': 'root',          
    'password': '?10101948SanMartin'  
}
# 🚨 ACTUALIZACIÓN: Usamos el nombre de archivo correcto
ARCHIVO_SQL = 'Ejercicio1_Fase4.sql'

# --- 2. FUNCIÓN DE UTILIDAD: LECTURA DEL ARCHIVO SQL ---

def leer_y_separar_consultas(archivo_path):
    """
    Lee un archivo SQL y separa cada consulta usando los comentarios de línea (--)
    como delimitadores para identificar el inicio de cada nueva consulta.
    """
    try:
        with open(archivo_path, 'r', encoding='utf-8') as f:
            contenido = f.read()
    except FileNotFoundError:
        print(f"❌ Error: Archivo '{archivo_path}' no encontrado. ¡Verifica la ruta!")
        return {}

    # Patrón para dividir por cualquier línea que comience con -- (comentario)
    # y capturar el comentario como identificador (clave del diccionario).
    # Este patrón es una versión robusta basada en la estructura que proporcionaste.
    consultas_divididas = re.split(r'(--\s*.*)', contenido)
    
    consultas = {}
    clave_actual = "Inicio" 
    
    for parte in consultas_divididas:
        parte = parte.strip()
        if not parte:
            continue
            
        if parte.startswith('--'):
            # El comentario es la nueva clave
            clave_actual = parte.replace('--', '').strip()
        else:
            # El texto es la consulta SQL
            if clave_actual != "Inicio" and parte.upper().startswith('SELECT'):
                # Usamos una versión limpia del comentario como clave del diccionario
                consultas[clave_actual] = parte
            
    return consultas

# --- 3. FUNCIÓN DE EJECUCIÓN ---

def ejecutar_multiples_consultas(consultas_dict):
    """Ejecuta todas las consultas del diccionario y almacena los resultados en DataFrames."""
    resultados = {}
    cnx = None
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        
        print(f"✅ Conexión establecida a la BD '{DB_CONFIG['database']}'.")
        print("--- INICIANDO EJECUCIÓN DE CONSULTAS ---")

        for nombre, query in consultas_dict.items():
            # Limpiamos la query de posibles saltos de línea y comentarios finales
            query_limpia = query.split(';')[0].strip() + ';'

            try:
                df = pd.read_sql(query_limpia, cnx)
                resultados[nombre] = df
                
                print(f"\n✅ Consulta: {nombre}")
                print("-" * (len(nombre) + 12))
                print(df)
                
            except Exception as e:
                print(f"\n❌ Error al ejecutar la consulta '{nombre}': {e}")
                
    except Error as e:
        print(f"\n❌ Error de conexión a MySQL: {e}")
        
    finally:
        if cnx and cnx.is_connected():
            cnx.close()
            print("\n✅ Conexión a MySQL cerrada.")
            
    return resultados

# --- 4. EJECUCIÓN DEL FLUJO PRINCIPAL ---

# 1. Leer y separar las consultas del archivo
consultas = leer_y_separar_consultas(ARCHIVO_SQL)

# 2. Ejecutar las consultas y obtener los resultados
dataframes_resultados = ejecutar_multiples_consultas(consultas)

✅ Conexión establecida a la BD 'Adalab_Peliculas'.
--- INICIANDO EJECUCIÓN DE CONSULTAS ---

✅ Consulta: ¿Cuántas películas tienen una duración superior a 120 minutos?
--------------------------------------------------------------------------
   Total_peliculas_largas
0                  2484.0

✅ Consulta: ¿Cuántas películas tienen contenido adulto?
-------------------------------------------------------
   Contenido_adultos
0             2884.0

✅ Consulta: ¿Cuál es la película más antigua registrada en la base de datos
---------------------------------------------------------------------------
         Titulo  Año_Lanzamiento
0  Citizen Kane             1941

✅ Consulta: Muestra el promedio de duración de las películas agrupado por género
--------------------------------------------------------------------------------
             Genero  Duracion_Promedio
0            Crimen           154.2857
1             Drama           126.2593
2            Acción           139.4444
3   Ciencia 

/var/folders/23/lk8jbvg92l12y_d7clg4mxtr0000gn/T/ipykernel_26512/3789162443.py:71: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_limpia, cnx)
/var/folders/23/lk8jbvg92l12y_d7clg4mxtr0000gn/T/ipykernel_26512/3789162443.py:71: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_limpia, cnx)
/var/folders/23/lk8jbvg92l12y_d7clg4mxtr0000gn/T/ipykernel_26512/3789162443.py:71: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_limpia, cnx)
/var/folders/23/lk8jbvg92l12y_d7cl

## COMENTARIOS ADICIONALES

En el ejericicio de arriba no he importado a la tabla la columna subtitulos porque entendía del enunciado que solo importabamos las cinco columnas principales 
Título
Año de lanzamiento
Duración (en minutos)
Género
Contenido para adultos (sí o no)

En el Data Frame df_peliculas tengo la columna subtitulos, añado esa columna a la BD para realizar la consulta que falta

No repite la conexión a My SQL, hay que ejecutar el codigo de arriba

In [ ]:
try:
    # 1. Crear el DataFrame limpio (usando df_peliculas)
    df_clean = df_peliculas.replace({np.nan: None, 'nan': None, 'NaN': None})

    # 2. Abrir la conexión
    cnx = mysql.connector.connect(**DB_CONFIG)
    mycursor = cnx.cursor()

    # 3. Añadir la columna Subtitulos en MySQL
    try:
        mycursor.execute("USE Adalab_Peliculas;")
        mycursor.execute("ALTER TABLE peliculas ADD COLUMN Subtitulos VARCHAR(255) NULL;")
        cnx.commit()
        print("✅ Columna 'Subtitulos' añadida a la tabla.")
    except Error as e:
        if "Duplicate column name" in str(e):
            print("Columna 'Subtitulos' ya existía, continuando con la actualización.")
        else:
            print(f"❌ Error al alterar la tabla: {e}")
            raise # Detener si hay un error grave

    # 4. Preparar la sentencia UPDATE
    # Usaremos el título como clave para la actualización
    query_update = """
    UPDATE peliculas
    SET Subtitulos = %s
    WHERE Titulo = %s;
    """

    datos_a_actualizar = []
    
    # 5. Iterar sobre df_clean para crear las tuplas de (Subtitulo_String, Titulo)
    for index, row in df_clean.iterrows():
        
        # Conversión de lista Python a string MySQL (ej. ['es', 'en'] -> 'es,en')
        subtitulos_lista = row.get('subtitulos')
        # Si es una lista, la unimos con comas; si no, usamos el valor tal cual (será None o NaN previamente limpiado)
        subtitulos_a_insertar = ','.join(subtitulos_lista) if isinstance(subtitulos_lista, list) else subtitulos_lista
        
        # Tupla para la actualización: (Nuevo_Valor_Subtitulos, Titulo_Referencia)
        tupla_update = (
            subtitulos_a_insertar,
            row['titulo']
        )
        datos_a_actualizar.append(tupla_update)

    print(f"✅ {len(datos_a_actualizar)} sentencias UPDATE preparadas.")

    # 6. Ejecutar la actualización masiva
    mycursor.executemany(query_update, datos_a_actualizar)
    cnx.commit()
    print(f"🎉 ÉXITO: {mycursor.rowcount} filas actualizadas con los subtítulos.")

except Error as e:
    print(f"❌ Error en la actualización de datos: {e}")

finally:
    if 'cnx' in locals() and cnx.is_connected():
        mycursor.close()
        cnx.close()
        print("Conexión cerrada.")

✅ Columna 'Subtitulos' añadida a la tabla.
✅ 100 sentencias UPDATE preparadas.
🎉 ÉXITO: 100 filas actualizadas con los subtítulos.
Conexión cerrada.


Hago la última consulta directamente en python, como he cerrado la conexion tengo que volver a abrirla solo para hacer la query

¿Cuántas películas incluyen subtítulos en español?

In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# --- Configuración de la Base de Datos ---
DB_CONFIG = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': '?10101948SanMartin',
    'database': 'Adalab_Peliculas' # Añadir la BD aquí es la mejor práctica
}

try:
    # 1. Conectar a la base de datos específica
    cnx = mysql.connector.connect(**DB_CONFIG)
    
    # 2. Crear el CURSOR
    mycursor = cnx.cursor()
    print('Conexión exitosa')

    # 3. Definir la query
    query = """
    SELECT
        COUNT(Titulo) as Total_Peliculas_SUbtitulos_Español
    FROM 
        peliculas
    WHERE 
        Subtitulos LIKE '%es%';
    """
    
    # 4. Ejecutar la query usando pd.read_sql
    # Ya está conectado a la BD Adalab_Peliculas pq la he incluido en el BD_CONFIG
    df_peliculas = pd.read_sql(query, cnx)
    
    # 5. Imprimir el resultado
    print('\nResultado de la consulta:')
    print(df_peliculas)
    
except Error as e:
    print('Error al conectar o ejecutar la consulta:', e)

finally:
    # 6. Cerrar la conexión si se abrió
    if 'cnx' in locals() and cnx.is_connected():
        mycursor.close()
        cnx.close()
        print('Conexión cerrada.')

Conexión exitosa

Resultado de la consulta:
   Total_Peliculas_SUbtitulos_Español
0                                 100
Conexión cerrada.


/var/folders/23/lk8jbvg92l12y_d7clg4mxtr0000gn/T/ipykernel_26512/2658469650.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_peliculas = pd.read_sql(query, cnx)
